# Overall Stats

In [1]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
import pandas as pd 

In [2]:
dfDeliveries = pd.read_csv("../data/IPL_Ball_by_Ball_2008_2022.csv") 
dfMatches = pd.read_csv("../data/IPL_Matches_2008_2022.csv")


Merge the dataframes

In [3]:
dfMerged = dfDeliveries.merge(dfMatches, on='ID')

In [10]:
dfMerged.head(5)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,...,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,BowlingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans


In [9]:
dfMerged.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'City', 'Date', 'Season',
       'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'method',
       'Player_of_Match', 'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2',
       'BowlingTeam'],
      dtype='object')

Adding the bowling team column

In [8]:
dfMerged['BowlingTeam'] = np.where(dfMerged['BattingTeam']==dfMerged['Team1'],
                                       dfMerged['Team2'],
                                       dfMerged['Team1'])

Cleaning the season column

In [13]:
dfMerged['Season'].unique()

array(['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015',
       '2014', '2013', '2012', '2011', '2010', '2009', '2008'],
      dtype=object)

In [12]:
dfMerged['Season'] = dfMerged['Season'].replace({'2020/21': '2020', '2009/10': '2010', '2007/08': '2008'})

batting group and bowling group

In [14]:
batgroup = dfMerged.groupby(['BattingTeam'])
bowlgroup = dfMerged.groupby(['BowlingTeam'])

batting Stats collection

In [15]:
batting_team_stats = pd.DataFrame(batgroup['ballnumber'].count()).rename(columns={'ballnumber':'balls_faced'})

In [16]:
batting_team_stats['matches_played'] = batgroup['ID'].nunique()

In [17]:
batting_team_stats['runs'] = batgroup['batsman_run'].sum()

In [20]:
batting_team_stats['4s'] = dfMerged[(dfMerged['batsman_run'] == 4) & (dfMerged['non_boundary'] == 0)].groupby('BattingTeam')['batsman_run'].count()

In [24]:
batting_team_stats['4s'].fillna(0,inplace=True)

In [22]:
batting_team_stats['6s'] = dfMerged[(dfMerged['batsman_run'] == 6) & (dfMerged['non_boundary'] == 0)].groupby('BattingTeam')['batsman_run'].count()


In [25]:
batting_team_stats['6s'].fillna(0,inplace=True)


In [27]:
batting_team_stats['1s'] = dfMerged[(dfMerged['batsman_run'] == 1) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
batting_team_stats['1s'].fillna(0,inplace=True)

In [29]:
batting_team_stats['2s'] = dfMerged[(dfMerged['batsman_run'] == 2) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
batting_team_stats['2s'].fillna(0,inplace=True)

In [33]:
batting_team_stats['0s'] = dfMerged[(dfMerged['batsman_run'] == 0) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
batting_team_stats['0s'].fillna(0,inplace=True)

In [50]:
batting_team_stats['dismissals'] = batgroup['isWicketDelivery'].sum()

In [56]:
batting_team_stats = batting_team_stats.drop('wickets_taken',axis=1)

In [57]:
batting_team_stats.head(5)

,balls_faced,matches_played,runs,4s,6s,1s,2s,dismissals,0s
BattingTeam,,,,,,,,,
Chennai Super Kings,25128,208,31734,2788,1269,9399,1623,1089,8663
Deccan Chargers,9034,75,10885,957,400,3387,563,484,3227
Delhi Capitals,7587,63,9691,912,347,2956,458,372,2519
Delhi Daredevils,18786,161,23022,2156,801,7011,1176,952,6595
Gujarat Lions,3566,30,4629,460,155,1360,233,188,1155


# bowling stats

In [35]:
bowling_team_stats = pd.DataFrame(bowlgroup['ballnumber'].count()).rename(columns={'ballnumber':'balls_bowled'})

In [37]:
bowling_team_stats['matches_played'] = bowlgroup['ID'].nunique()

In [39]:
bowling_team_stats['runs_conceded'] = batgroup['total_run'].sum()

In [41]:
bowling_team_stats['4s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 4) & (dfMerged['non_boundary'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['4s_conceded'].fillna(0,inplace=True)

In [43]:
bowling_team_stats['6s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 6) & (dfMerged['non_boundary'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['6s_conceded'].fillna(0,inplace=True)

In [44]:
bowling_team_stats['1s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 1) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['1s_conceded'].fillna(0,inplace=True)

In [45]:
bowling_team_stats['2s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 2) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['2s_conceded'].fillna(0,inplace=True)

In [46]:
bowling_team_stats['0s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 0) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['0s_conceded'].fillna(0,inplace=True)

In [58]:
bowling_team_stats['wickets_taken'] = bowlgroup['isWicketDelivery'].sum()

In [60]:
dfMerged['extra_type'].unique()

array([nan, 'legbyes', 'wides', 'byes', 'noballs', 'penalty'],
      dtype=object)

In [66]:
bowling_team_stats['noballs_conceded'] = dfMerged[dfMerged['extra_type'] == 'noballs'].groupby('BowlingTeam')['total_run'].count()
bowling_team_stats['noballs_conceded'].fillna(0,inplace=True)

In [69]:
dfMerged['kind'].unique()

array([nan, 'caught', 'caught and bowled', 'run out', 'bowled', 'stumped',
       'lbw', 'hit wicket', 'retired hurt', 'retired out',
       'obstructing the field'], dtype=object)

In [70]:
bowling_team_stats['catches_taken'] = dfMerged[dfMerged['kind'] == 'caught'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['catches_taken'].fillna(0,inplace= True)

In [72]:
bowling_team_stats['run_out_taken'] = dfMerged[dfMerged['kind'] == 'run out'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['run_out_taken'].fillna(0,inplace= True)

In [73]:
bowling_team_stats['bowled_taken'] = dfMerged[dfMerged['kind'] == 'bowled'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['bowled_taken'].fillna(0,inplace= True)

In [74]:
bowling_team_stats['stumped_taken'] = dfMerged[dfMerged['kind'] == 'stumped'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['stumped_taken'].fillna(0,inplace= True)

In [75]:
bowling_team_stats['lbw_taken'] = dfMerged[dfMerged['kind'] == 'lbw'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['lbw_taken'].fillna(0,inplace= True)

In [76]:
bowling_team_stats.head(5)

,balls_bowled,matches_played,runs_conceded,4s_conceded,6s_conceded,1s_conceded,2s_conceded,0s_conceded,wickets_taken,legbyes_conceded,wides_conceded,byes_conceded,noballs_conceded,catches_taken,run_out_taken,bowled_taken,stumped_taken,lbw_taken
BowlingTeam,,,,,,,,,,,,,,,,,,
Chennai Super Kings,24851,208,33393,2714,1157,9168,1603,8848,1290,371,782,49,81,823,109,197,34,83
Deccan Chargers,9039,75,11463,1013,405,3181,579,3306,446,166,280,31,49,271,38,82,17,26
Delhi Capitals,7730,63,10145,847,396,3049,432,2627,405,92,229,14,29,275,22,63,13,21
Delhi Daredevils,18725,160,24296,2180,870,6953,1139,6521,912,324,550,39,86,528,116,173,30,44
Gujarat Lions,3545,30,4862,453,190,1354,285,1097,151,55,91,7,7,94,18,21,3,11


combining batting and bowling stats

In [77]:
combined_team_stats = pd.merge(batting_team_stats, bowling_team_stats, left_on='BattingTeam', right_on='BowlingTeam', how='outer')


combined_team_stats


,balls_faced,matches_played_x,runs,4s,6s,1s,2s,dismissals,0s,balls_bowled,...,wickets_taken,legbyes_conceded,wides_conceded,byes_conceded,noballs_conceded,catches_taken,run_out_taken,bowled_taken,stumped_taken,lbw_taken
0,25128,208,31734,2788,1269,9399,1623,1089,8663,24851,...,1290,371,782,49,81,823,109,197,34,83
1,9034,75,10885,957,400,3387,563,484,3227,9039,...,446,166,280,31,49,271,38,82,17,26
2,7587,63,9691,912,347,2956,458,372,2519,7730,...,405,92,229,14,29,275,22,63,13,21
3,18786,161,23022,2156,801,7011,1176,952,6595,18725,...,912,324,550,39,86,528,116,173,30,44
4,3566,30,4629,460,155,1360,233,188,1155,3545,...,151,55,91,7,7,94,18,21,3,11
5,1971,16,2517,250,79,794,121,82,622,1907,...,101,22,55,3,6,64,10,12,3,9
6,22646,190,28541,2626,1075,8296,1498,1158,7866,22483,...,1076,362,720,52,100,641,92,218,26,65
7,1582,14,1758,170,53,570,85,86,595,1614,...,74,29,52,2,7,35,12,16,1,8
8,26192,223,32330,3021,1227,9545,1507,1318,9378,26355,...,1275,405,733,87,114,722,123,244,50,96
9,1840,15,2405,188,115,711,118,89,595,1843,...,100,27,72,2,6,71,7,15,1,4


In [82]:
combined_team_stats.shape

(18, 27)

In [84]:
combined_team_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   balls_faced       18 non-null     int64
 1   runs              18 non-null     int64
 2   4s                18 non-null     int64
 3   6s                18 non-null     int64
 4   1s                18 non-null     int64
 5   2s                18 non-null     int64
 6   dismissals        18 non-null     int64
 7   0s                18 non-null     int64
 8   balls_bowled      18 non-null     int64
 9   runs_conceded     18 non-null     int64
 10  4s_conceded       18 non-null     int64
 11  6s_conceded       18 non-null     int64
 12  1s_conceded       18 non-null     int64
 13  2s_conceded       18 non-null     int64
 14  0s_conceded       18 non-null     int64
 15  wickets_taken     18 non-null     int64
 16  legbyes_conceded  18 non-null     int64
 17  wides_conceded    18 non-null     int

In [83]:
# Keep only one of the 'matches_played' columns
combined_team_stats['matches_played'] = combined_team_stats['matches_played_x'].combine_first(combined_team_stats['matches_played_y'])

# Drop the unnecessary 'matches_played_x' and 'matches_played_y' columns
combined_team_stats = combined_team_stats.drop(['matches_played_x', 'matches_played_y'], axis=1)

In [85]:
combined_team_stats.to_csv('../derived/combined_team_stats_overall.csv',index=False)